In [1]:
import pandas as pd
import glob
import os
import numpy as np
from scipy.stats import f_oneway, kruskal, ttest_ind

In [2]:
modality_list = ['T1CE', 'T1W', 'T2W', 'T2F']

In [3]:
#cd '/home/raghuram/Desktop/radiomics/expt/data/T2F'

In [4]:
csv_file_list = sorted(os.listdir(os.getcwd()))

In [99]:
base_results_folder = '/home/raghuram/Desktop/radiomics/expt/results'
data_folder = '/home/raghuram/Desktop/radiomics/expt/data'

In [93]:
#results_folder = '/home/raghuram/Desktop/radiomics/expt/results/T2F/scanner_manufacturer/anova/'
alpha = 0.05

In [102]:
for modality in modality_list:
    csv_file_list = sorted(os.listdir(os.path.join(data_folder, modality)))
    results_folder = os.path.join(base_results_folder, modality, 'mag_field_strength', 't_test')
    for csv_file in csv_file_list:
        expt_number = csv_file.split('_')[1].split('.')[0]
        out_df = {}
        df = pd.read_csv(csv_file)
        df = df[df['Magnetic Strength'] != 1.16]
        df.dropna(subset=['Magnetic Strength'], inplace=True)
        df.loc[df['Magnetic Strength'] == 1.493806, ['Magnetic Strength']] = 1.5
    #     out_df['scale'] = df['parameters_Scale']
    #     out_df['algo'] = df['parameters_Algo']
    #     out_df['ng'] = df['parameters_Ng']
        df.drop(columns =  ['Filename', 'parameters_Scale', 'parameters_Algo', 'parameters_Ng', 'Scanner Manufacturer',
                             'codel_status', 'death_days', 'idh_status'], 
                inplace=True)
        one_five = df[df['Magnetic Strength'] == 1.5]
        one_five.drop(columns='Magnetic Strength', inplace=True)
        three = df[df['Magnetic Strength'] == 3]
        three.drop(columns = 'Magnetic Strength', inplace=True)
    #     other = df[df['Magnetic Strength'] == 1.493806]
    #     other.drop(columns = 'Magnetic Strength', inplace=True)

        df.drop(columns ='Magnetic Strength', inplace=True)
        column_names = list(df.columns)
        t, p_value  = ttest_ind(one_five, three)
        #F, p_value  = f_oneway(one_five, three, other)
        out_df['feature_names'] = column_names
        #out_df['F'] = list(F)
        out_df['t'] = list(t)
        out_df['p_value'] = list(p_value)
        out_df = pd.DataFrame.from_dict(out_df)
        status = ['REJECTED' if p_<alpha else 'NOT_REJECTED' for p_ in p_value]
        out_df['status'] = status
        out_df.to_csv(os.path.join(results_folder, expt_number+'t_test'+'.csv'), index=False)

    print ('Finished with modality {}'.format(modality))

Finished with modality T1CE
Finished with modality T1W
Finished with modality T2W
Finished with modality T2F


In [101]:
for modality in modality_list:
    csv_file_list = sorted(os.listdir(os.path.join(data_folder, modality)))
    results_folder = os.path.join(base_results_folder, modality, 'scanner_manufacturer', 'anova')
    for csv_file in csv_file_list:
        expt_number = csv_file.split('_')[1].split('.')[0]
        df = pd.read_csv(csv_file)
        df.dropna(subset=['Scanner Manufacturer'], inplace=True)
        df['Scanner Manufacturer'] = df['Scanner Manufacturer'].str.strip()
    #     print(df['Scanner Manufacturer'])
    #     break
        df.drop(columns = [ 'Filename', 'parameters_Scale', 'parameters_Algo', 'parameters_Ng', 'Magnetic Strength',
                              'codel_status', 'death_days', 'idh_status'], inplace=True)
        df = df[df['Scanner Manufacturer'] != 'Hitachi Medical Corporation']
        GE = df[df['Scanner Manufacturer'] == 'GE MEDICAL SYSTEMS']
        GE.drop(columns = ['Scanner Manufacturer'], inplace=True)
        Philips = df[(df['Scanner Manufacturer'] == 'Philips Medical Systems') | \
                     (df['Scanner Manufacturer'] == 'Philips Healthcare')]
        Philips.drop(columns = ['Scanner Manufacturer'], inplace=True)
        Siemens = df[(df['Scanner Manufacturer'] == 'SIEMENS') | \
                     (df['Scanner Manufacturer'] == 'Siemens')]
        Siemens.drop(columns = ['Scanner Manufacturer'], inplace=True)
        #non_GE = df[df['Scanner Manufacturer'] != 'GE MEDICAL SYSTEMS']
        #print(non_GE.describe())
        #non_GE.drop(columns = ['Scanner Manufacturer'], inplace=True)
        df.drop(columns = ['Scanner Manufacturer'], inplace=True)
        column_names = list(df.columns)
        out_df = {}
        F, p_value = f_oneway(GE, Philips, Siemens)
        out_df['feature_names'] = column_names
        out_df['F'] = list(F)
        out_df['p_value'] = list(p_value)
        out_df = pd.DataFrame.from_dict(out_df)
        status = ['REJECTED' if p_<alpha else 'NOT_REJECTED' for p_ in p_value]
        out_df['status'] = status
        out_df.to_csv(os.path.join(results_folder, expt_number+'anova'+'.csv'), index=False)

    print ('Finished with modality {}'.format(modality))

Finished with modality T1CE
Finished with modality T1W
Finished with modality T2W
Finished with modality T2F


In [3]:
filename = 'Features_Experiment'
discard_range = [str(i) for i in range(33, 41)]
patient_id = 'TCGA-HT-7692_T1CE.mat'

In [4]:
for expt_num in discard_range:
    filename_appended = filename+expt_num+'.csv'
    df = pd.read_csv(filename_appended)
    df = df[df['Filename'] != patient_id]
    df.to_csv(filename_appended)